In [1]:
!pip install pdfminer.six
!pip install -U pip setuptools wheel
!pip install -U spacy
# !python -m spacy download en_core_web_sm
!python -m spacy download en_core_web_trf

     |████████████████████████████████| 5.6 MB 8.0 MB/s 
     |████████████████████████████████| 3.6 MB 43.1 MB/s 
     |████████████████████████████████| 2.1 MB 8.2 MB/s 
     |████████████████████████████████| 1.1 MB 54.2 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.0/452.0 KB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 KB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.2/628.2 KB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 KB 6.0 MB/s eta 0:00:00
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installa

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
from pdfminer.high_level import extract_text

def extract_text_from_pdf(pdf_path):
    return extract_text(pdf_path)


In [69]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_trf")
doc = nlp(extract_text_from_pdf("/content/Aberdeen_Bernice Tan_Compliance Assistant Manager.pdf"))
doc1 = nlp(extract_text_from_pdf("/content/3Carrie.pdf"))


/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
Token indices sequence length is longer than the specified maximum sequence length for this model (567 > 512). Running this sequence through the model will result in indexing errors


In [70]:
# initialize matcher with a vocab
matcher = Matcher(nlp.vocab)

def extract_name(resumetext,matcher):
    nlp_text = nlp(resumetext)
    
    # First name and Last name - Proper Nouns
    pattern1 = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
    pattern2 = [{'POS': 'PROPN','OP':'*'}]
    matcher.add('Name',[pattern1])
    matcher.add('First Name',[pattern2])
    
    matches = matcher(nlp_text)
    
    for match_id, start, end in matches:
        span = nlp_text[start:end]
        return span.text
#Doc = doc[0:5]
subdoc = nlp(doc.text[0:20])
#print(subdoc)
extract_name(subdoc,matcher)

/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


'Bernice Tan'

In [67]:

list_of_per=[]

#people = [ee for ee in doc.ents if ee.label_ == 'PERSON']


for entity in doc.ents:
 if entity.label_=='PERSON':
    list_of_per.append(entity.text)
print(list_of_per)


['Daniel Schwarz']


In [66]:
import re

doctext = nlp(extract_text("/content/BCG_Daniel Schwarz_Consultant.pdf"))
#print(doctext)

spacy_nlp  = spacy.load('en_core_web_trf')

# parse text into spacy document
doc = spacy_nlp(doctext)

# create sets to hold words
named_entities = set()
money_entities = set()
organization_entities = set()
location_entities = set()
time_indicator_entities = set()

for i in doc.ents:
    entry = str(i.lemma_).lower()
    re.sub(str(i).lower(), "", str(doctext))
    # Time indicator entities detection
    if i.label_ in ["TIM", "DATE"]:
        time_indicator_entities.add(entry)
    # money value entities detection
    elif i.label_ in ["MONEY"]:
        money_entities.add(entry)
    # organization entities detection
    elif i.label_ in ["ORG"]:
        organization_entities.add(entry)
    # Geographical and Geographical entities detection
    elif i.label_ in ["GPE", "GEO"]:
        location_entities.add(entry)
    # extract artifacts, events and natural phenomenon from text
    elif i.label_ in ["ART", "EVE", "NAT", "PERSON"]:
        named_entities.add(entry.title())

print(f"named entities - {named_entities}")
print(f"money entities - {money_entities}")
print(f"location entities - {location_entities}")
print(f"time indicator entities - {time_indicator_entities}")
print(f"organization entities - {organization_entities}")

/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


named entities - {'Daniel Schwarz'}
money entities - {'£1bn cost', 'us$10mn+ debt holder', '5mn', '£2m'}
location entities - {'singapore', 'uk', 'nepal'}
time indicator entities - {'2013 - 2014', '2010 - 2012', '2007', '3 year', 'three month', '2007 - 2009', '2016 - present', '2015', 'december 2015', '2010 - 2013'}
organization entities - {'durham university', 'insead business school', 'the boston consulting group', 'hsbc middle east', 'icaew - institute of chartered accountants', 'aca', 'standard chartered bank', 'project management office'}


In [65]:
for entity in doc.ents:
  print(entity)
  print(entity.label_)
#     list_of_per.append(entity.text)
# print(list_of_skills)

Bernice Tan
PERSON
Singapore
GPE
Aberdeen Asset Management
ORG
Aberdeen Asset Management
ORG
January 2017  
DATE
3 months
DATE
Aberdeen Asset Management
ORG
January 2015  
DATE
December 2016
DATE
2 years
DATE
Aberdeen Asset Management
ORG
July 2012  
DATE
December 2014
DATE
2 years 6 months
DATE
HSBC Global Asset Management
ORG
June 2011  
DATE
September 2011
DATE
4 months
DATE
University of London International Programmes
ORG
Bachelor’s Degree
WORK_OF_ART
2009 - 2012
DATE
Anglo-Chinese
NORP
2007 - 2008
DATE
Bernice Tan
PERSON
Singapore
GPE
Aberdeen Asset Management
ORG
1
CARDINAL
3/7/2017
DATE
Ayyaz Ahmad
PERSON
